# Validation

OpenFisca-UK runs unit and integration tests on each new version (see [here](https://github.com/PSLmodels/openfisca-uk/tree/master/tests)).
In addition, the table below shows the aggregates produced by the model for the major taxes and benefits, and comparisons with UKMOD (latest [country report](https://www.iser.essex.ac.uk/research/publications/working-papers/cempa/cempa7-20.pdf)) and official sources.[^1]
UKMOD and administrative sources refer to 2018, and OpenFisca-UK is simulated on policy at the end of 2018.
Numbers are in billions of pounds.

[^1]: From the UKMOD country report: unless otherwise specified: Department for Work and Pensions https://www.gov.uk/government/publications/benefit-expenditure-and-caseload-tables-2018 ; Best Start Grant: https://www2.gov.scot/Topics/Statistics/Browse/Social-Welfare/SocialSecurityforScotland/BSGJune2019; Child tax credit and working tax credit: HMRC statistics 
https://www.gov.uk/government/statistics/child-and-working-tax-credits-statistics-finalised-annual-awards-2016-to-2017; Scottish Child Payment: Scottish Fiscal Commission https://www.fiscalcommission.scot/forecast/supplementary-costing-scottish-child-payment; Scottish Child Winter Heating Assistance: Scottish Fiscal Commission 
https://www.fiscalcommission.scot/forecast/supplementary-costing-child-winter-heating-assistance; Income tax: HMRC statistics https://www.gov.uk/government/statistics/income-tax-liabilities-statistics-tax-year-2014-to-2015-to-tax-year-2017-to-2018; National Insurance Contributions: ONS Blue Book Table 5.2.4s 

In [1]:
from openfisca_uk import Microsimulation
from openfisca_uk_data import FRS
import pandas as pd

sim = Microsimulation(year=2018, dataset=FRS)

VARIABLES = dict(
    income_tax="Income Tax",
    national_insurance="Non-employer National Insurance",
    employee_NI_class_1="Employee Class 1 NI",
    employer_NI_class_1="Employer Class 1 NI",
    NI_class_4="Class 2 & 4 NI",
    universal_credit="Universal Credit",
    tax_credits="Tax Credits",
    housing_benefit="Housing Benefit",
    state_pension="State Pension",
)


v = pd.DataFrame(columns=["ukmod", "external"])
v.loc["income_tax"] = 163.7, 190.0
v.loc["national_insurance"] = 64.0, 55.7
v.loc["employee_NI_class_1"] = 55.7, 52.0
v.loc["employer_NI_class_1"] = 77.8, 79.2
v.loc["NI_class_4"] = 5.0, 3.7
v.loc["universal_credit"] = 11.7, 8.1
v.loc["tax_credits"] = 13.9, 22.8
v.loc["housing_benefit"] = 15.1, 20.7
v.loc["state_pension"] = 86.4, 85.5

v["of"] = (
    sim.df(list(VARIABLES.keys()), map_to="household", period=2018).sum() / 1e9
)

v["of_ukmod"] = round(v.of / v.ukmod, 2)
v["of_external"] = round(v.of / v.external, 2)
v.of = v.of.round(1)
COLUMN_RENAMES = dict(
    of="OpenFisca-UK",
    ukmod="UKMOD",
    external="External",
    of_ukmod="OpenFisca-UK / UKMOD",
    of_external="OpenFisca-UK / External",
)
v[COLUMN_RENAMES.keys()].rename(index=VARIABLES, columns=COLUMN_RENAMES)

,OpenFisca-UK,UKMOD,External,OpenFisca-UK / UKMOD,OpenFisca-UK / External
Income Tax,168.2,163.7,190.0,1.03,0.89
Non-employer National Insurance,62.3,64.0,55.7,0.97,1.12
Employee Class 1 NI,56.7,55.7,52.0,1.02,1.09
Employer Class 1 NI,79.3,77.8,79.2,1.02,1.00
Class 2 & 4 NI,5.1,5.0,3.7,1.01,1.37
Universal Credit,7.8,11.7,8.1,0.67,0.97
Tax Credits,14.7,13.9,22.8,1.05,0.64
Housing Benefit,11.5,15.1,20.7,0.76,0.55
State Pension,86.4,86.4,85.5,1.00,1.01


## Aggregate tables

OpenFisca-UK uprates input FRS data: below are comparisons between the aggregates calculated by OpenFisca-UK, UKMOD and external sources.

### Aggregates in full

In [2]:
import numpy as np

_ = np.nan
VARIABLES = [
    "income_tax",
    "total_NI",
    "universal_credit",
    "working_tax_credit",
    "child_tax_credit",
    "child_benefit",
    "housing_benefit",
    "pension_credit",
    "income_support",
    "JSA_income",
]

df = pd.concat(
    [
        (sim.df(VARIABLES, map_to="household", period=year).sum() / 1e9).round(
            1
        )
        for year in range(2018, 2022)
    ],
    axis=1,
)
df.columns = list(range(2018, 2022))
df.index = [
    sim.simulation.tax_benefit_system.variables[var].label for var in df.index
]
df
ukmod_df = pd.DataFrame(
    {
        "Income Tax": [163.7, 165.9, 165.0, 173.9],
        "NI (total)": [138.6, 144.2, 141.6, 148.0],
        "Universal Credit": [11.7, 24.8, 41.3, 40.4],
        "Working Tax Credit": [2.5, 1.6, 1.3, 0.6],
        "Child Tax Credit": [11.4, 7.1, 4.4, 2.8],
        "Housing Benefit": [15.1, 11.0, 8.6, 7.5],
        "Child Benefit": [11.5, 11.4, 11.6, 11.6],
        "Pension Credit": [4.1, 3.6, 3.6, 2.9],
        "Income Support": [_, _, _, _],
        "JSA (income-based)": [_, _, _, _],
    }
).T
ukmod_df.columns = list(range(2018, 2022))
# source: https://www.microsimulation.ac.uk/wp-content/uploads/2020/10/cempa7-20.pdf#page=130
# where missing, UKMOD does not separate benefits and therefore figures cannot be obtained
external_df = pd.DataFrame(
    {
        "Income Tax": [
            192.5,
            193.6,
            188.2,
            203.6,
        ],  # source: https://obr.uk/forecasts-in-depth/tax-by-tax-spend-by-spend/income-tax/
        "NI (total)": [
            137.4,
            145.0,
            140.8,
            145.9,
        ],  # source: https://obr.uk/forecasts-in-depth/tax-by-tax-spend-by-spend/national-insurance-contributions-nics/
        "Universal Credit": [
            8.1,
            18.4,
            38.2,
            41.3,
        ],  # source: https://www.gov.uk/government/publications/benefit-expenditure-and-caseload-tables-2021
        "Working Tax Credit": [5.0, _, _, _],
        "Child Tax Credit": [17.7, _, _, _],
        "Housing Benefit": [20.7, 18.4, 17.3, 16.9],
        "Child Benefit": [
            11.6,
            11.6,
            11.9,
            12.1,
        ],  # source: https://obr.uk/docs/dlm_uploads/Welfare_trends_report_December_2019.pdf#page=16
        "Pension Credit": [5.1, 5.1, 5.3, 5.3],
        "Income Support": [1.8, 1.4, 1.1, 0.8],
        "JSA (income-based)": [1.1, 0.6, 0.4, 0.3],
    }
).T
external_df.columns = list(range(2018, 2022))
pd.concat(
    [df, ukmod_df, external_df],
    axis=1,
    keys=["OpenFisca-UK", "UKMOD", "External"],
).fillna("")

OpenFisca-UK                       UKMOD                \
                           2018   2019   2020   2021   2018   2019   2020   
Income Tax                168.2  174.2  168.0  178.9  163.7  165.9  165.0   
NI (total)                141.6  146.1  146.6  150.1  138.6  144.2  141.6   
Universal Credit            7.8   15.2   22.6   29.6   11.7   24.8   41.3   
Working Tax Credit          2.7    1.9    1.3    1.6    2.5    1.6    1.3   
Child Tax Credit           11.9    8.6    6.0    5.1   11.4    7.1    4.4   
Child Benefit              11.9   11.9   11.9   12.2   11.5   11.4   11.6   
Housing Benefit            11.5    8.4    5.9    4.8   15.1   11.0    8.6   
Pension Credit              2.2    2.6    2.8    3.2    4.1    3.6    3.6   
Income Support              2.7    2.0    1.5    1.2                        
JSA (income-based)          1.0    0.8    0.5    0.5                        

                          External                       
                     2021     2018   2019   2020   2021  
Income Tax          173.9    192.5  193.6  188.2  203.6  
NI (total)          148.0    137.4  145.0  140.8  145.9  
Universal Credit     40.4      8.1   18.4   38.2   41.3  
Working Tax Credit    0.6      5.0                       
Child Tax Credit      2.8     17.7                       
Child Benefit        11.6     11.6   11.6   11.9   12.1  
Housing Benefit       7.5     20.7   18.4   17.3   16.9  
Pension Credit        2.9      5.1    5.1    5.3    5.3  
Income Support                 1.8    1.4    1.1    0.8  
JSA (income-based)             1.1    0.6    0.4    0.3

### Differences

In [3]:
pd.concat(
    [
        df,
        (df - ukmod_df).round(1).fillna(""),
        (df - external_df).round(1).fillna(""),
    ],
    axis=1,
    keys=["OpenFisca-UK", "UKMOD Difference", "External Difference"],
).fillna("")

OpenFisca-UK                      UKMOD Difference       \
                           2018   2019   2020   2021             2018 2019   
Income Tax                168.2  174.2  168.0  178.9              4.5  8.3   
NI (total)                141.6  146.1  146.6  150.1              3.0  1.9   
Universal Credit            7.8   15.2   22.6   29.6             -3.9 -9.6   
Working Tax Credit          2.7    1.9    1.3    1.6              0.2  0.3   
Child Tax Credit           11.9    8.6    6.0    5.1              0.5  1.5   
Child Benefit              11.9   11.9   11.9   12.2              0.4  0.5   
Housing Benefit            11.5    8.4    5.9    4.8             -3.6 -2.6   
Pension Credit              2.2    2.6    2.8    3.2             -1.9 -1.0   
Income Support              2.7    2.0    1.5    1.2                         
JSA (income-based)          1.0    0.8    0.5    0.5                         

                               External Difference                    
                    2020  2021                2018  2019  2020  2021  
Income Tax           3.0   5.0               -24.3 -19.4 -20.2 -24.7  
NI (total)           5.0   2.1                 4.2   1.1   5.8   4.2  
Universal Credit   -18.7 -10.8                -0.3  -3.2 -15.6 -11.7  
Working Tax Credit   0.0   1.0                -2.3                    
Child Tax Credit     1.6   2.3                -5.8                    
Child Benefit        0.3   0.6                 0.3   0.3   0.0   0.1  
Housing Benefit     -2.7  -2.7                -9.2 -10.0 -11.4 -12.1  
Pension Credit      -0.8   0.3                -2.9  -2.5  -2.5  -2.1  
Income Support                                 0.9   0.6   0.4   0.4  
JSA (income-based)                            -0.1   0.2   0.1   0.2